In [3]:
# === Імпорт бібліотек та допоміжні функції ===
import numpy as np
from time import perf_counter

# Функція для тестування часу виконання
def bench(fn, *args, repeat=3, **kwargs):
    best = float("inf"); out = None
    for _ in range(repeat): 
        t0 = perf_counter()
        res = fn(*args, **kwargs)
        dt = perf_counter() - t0
        if dt < best:
            best, out = dt, res
    return out, best


# ==================================================================
# ВПРАВА 2. Шахматна матриця n×n (0/1), починаючи з 1 в [0,0]
# Варіант: NumPy + цикли
# ==================================================================
def task_2_numpy(n: int):
    a = np.zeros((n, n), dtype=int)
    a[::2, ::2] = 1
    a[1::2, 1::2] = 1
    return a

def task_2_loops(n: int):
    a = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(n):
            a[i, j] = 1 if (i + j) % 2 == 0 else 0
    return a


# ==================================================================
# ВПРАВА 4. Матриця n×m: перший ряд — значення 0…m-1, інші — нулі
# ==================================================================
def task_4_numpy(n: int, m: int):
    a = np.zeros((n, m), dtype=int)
    a[0] = np.arange(m)
    return a

def task_4_loops(n: int, m: int):
    a = np.zeros((n, m), dtype=int)
    for j in range(m):
        a[0, j] = j
    return a


# ==================================================================
# ВПРАВА 5. В парних рядках — одиниці, в непарних — нулі
# ==================================================================
def task_5_numpy(n: int):
    a = np.zeros((n, n), dtype=int)
    a[::2, :] = 1
    return a

def task_5_loops(n: int):
    a = np.zeros((n, n), dtype=int)
    for i in range(n):
        val = 1 if i % 2 == 0 else 0
        for j in range(n):
            a[i, j] = val
    return a


# ==================================================================
# ВПРАВА 8. Порахувати кількість нулів та не-нулів у векторі
# ==================================================================
def task_8_numpy(arr):
    zeros = np.count_nonzero(arr == 0)
    nonzeros = arr.size - zeros
    return zeros, nonzeros

def task_8_loops(arr):
    zeros = 0
    for x in arr:
        if x == 0:
            zeros += 1
    return zeros, len(arr) - zeros


# ==================================================================
# ВПРАВА 9. Масив значень від n до 0
# ==================================================================
def task_9_numpy(n: int):
    return np.arange(n, -1, -1)

def task_9_loops(n: int):
    out = [i for i in range(n, -1, -1)]
    return np.array(out)


# ==================================================================
# ВПРАВА 12. Матриця одиниць, але рамка з нулів
# ==================================================================
def task_12_numpy(n: int):
    a = np.ones((n, n), dtype=float)
    a[0, :] = a[-1, :] = a[:, 0] = a[:, -1] = 0.0
    return a

def task_12_loops(n: int):
    a = np.ones((n, n), dtype=float)
    for i in range(n):
        a[0, i] = a[-1, i] = a[i, 0] = a[i, -1] = 0.0
    return a


# ==================================================================
# ВПРАВА 13. Дошка 8×8 у шаховому порядку (0/1)
# ==================================================================
def task_13_numpy():
    base = np.array([[0,1],[1,0]], dtype=int)
    return np.tile(base, (4,4))

def task_13_loops():
    a = np.zeros((8,8), dtype=int)
    for i in range(8):
        for j in range(8):
            a[i,j] = (i + j + 1) % 2
    return a


# ==================================================================
# ВПРАВА 15. В парних стовпчиках — 1, у непарних — 0
# ==================================================================
def task_15_numpy(n: int):
    a = np.zeros((n, n), dtype=int)
    a[:, 1::2] = 1
    return a

def task_15_loops(n: int):
    a = np.zeros((n, n), dtype=int)
    for j in range(n):
        val = 1 if j % 2 == 1 else 0
        for i in range(n):
            a[i, j] = val
    return a


# ==================================================================
# ВПРАВА 19. Вектор з n елементів рівномірно на (0,1)
# ==================================================================
def task_19_numpy(n: int):
    v = np.linspace(0, 1, n+2, endpoint=False)[1:]
    return np.round(v, 3)

def task_19_loops(n: int):
    step = 1/(n+1)
    out = []
    for i in range(1, n+1):
        out.append(round(i*step, 3))
    return np.array(out)


# ==================================================================
# СЛАУ (4 рівняння) через 4 методи та порівняння
# ==================================================================
A = np.array([
    [0, 1, -3,  4],
    [1, 0, -2,  3],
    [3, 2,  0, -5],
    [4, 3, -5,  0]
], dtype=float)

b = np.array([-5, -4, 12, 5], dtype=float)

# Метод: обернена матриця
def solve_inv(A, b):
    return np.linalg.inv(A) @ b

# Метод: numpy.linalg.solve
def solve_solve(A, b):
    return np.linalg.solve(A, b)

# Метод Крамера
def solve_cramer(A, b):
    detA = np.linalg.det(A)
    x = np.zeros(len(b))
    for i in range(A.shape[1]):
        Ai = A.copy()
        Ai[:, i] = b
        x[i] = np.linalg.det(Ai) / detA
    return x

# Метод Гаусса на циклах
def solve_gauss_loops(A, b):
    A = A.astype(float).copy()
    b = b.astype(float).copy()
    n = len(b)
    for i in range(n):
        piv = i + np.argmax(np.abs(A[i:, i]))
        A[[i, piv]] = A[[piv, i]]
        b[i], b[piv] = b[piv], b[i]
        factor = A[i, i]
        A[i, i:] /= factor
        b[i] /= factor
        for k in range(i+1, n):
            c = A[k, i]
            A[k, i:] -= c * A[i, i:]
            b[k] -= c * b[i]
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = b[i] - np.dot(A[i, i+1:], x[i+1:])
    return x

# Функція для порівняння рішень
def compare_system_solvers():
    sol_inv,  t_inv  = bench(solve_inv,   A, b)
    sol_solve,t_solve= bench(solve_solve, A, b)
    sol_cr,   t_cr   = bench(solve_cramer,A, b)
    sol_gauss,t_gauss= bench(solve_gauss_loops, A, b)

    print("Рішення (solve):", np.round(sol_solve,6))
    print("inv@b збігається:", np.allclose(sol_inv,  sol_solve))
    print("Крамер збігається:", np.allclose(sol_cr,   sol_solve))
    print("Гаусс збігається :", np.allclose(sol_gauss,sol_solve))
    print(f"Час: inv={t_inv:.6f}  solve={t_solve:.6f}  cramer={t_cr:.6f}  gauss={t_gauss:.6f}")


# ==================================================================
# ВПРАВА 3. Матричний вираз (A^2 − B^2)·(A + B)
# ==================================================================
A3 = np.array([[ 7,  2, 0],
               [-7, -2, 1],
               [ 1,  1, 0]], dtype=float)

B3 = np.array([[0, 2, 3],
               [1, 0,-2],
               [3, 1, 1]], dtype=float)

def matmul_loops(X, Y):
    n, m = X.shape[0], Y.shape[1]
    Z = np.zeros((n, m), dtype=float)
    for i in range(n):
        for j in range(m):
            s = 0.0
            for k in range(X.shape[1]):
                s += X[i,k]*Y[k,j]
            Z[i,j] = s
    return Z

def matrix_expression_numpy():
    return (A3@A3 - B3@B3) @ (A3 + B3)

def matrix_expression_loops():
    A2 = matmul_loops(A3, A3)
    B2 = matmul_loops(B3, B3)
    left = A2 - B2
    right = A3 + B3
    return matmul_loops(left, right)

def compare_matrix_expr():
    res_np, t_np   = bench(matrix_expression_numpy)
    res_lp, t_loops= bench(matrix_expression_loops)
    print("allclose:", np.allclose(res_np, res_lp))
    print(f"Час: numpy={t_np:.6f}  loops={t_loops:.6f}")
    print("Результат:\n", np.round(res_np, 3))


# ==================================================================
# Тестові запускі (можна залишити або забрати)
# ==================================================================
if __name__ == "__main__":
    print("task_2:", np.allclose(task_2_numpy(5), task_2_loops(5)))
    print("task_4:\n", task_4_numpy(3,4))
    print("task_5:\n", task_5_numpy(5))
    arr = np.array([3,4,0,9,8,2,4,0,8,4,0])
    print("task_8:", task_8_numpy(arr))
    print("task_9:", task_9_numpy(10))
    print("task_12:\n", task_12_numpy(4))
    print("task_13:\n", task_13_numpy())
    print("task_15:\n", task_15_numpy(5))
    print("task_19 numpy:", task_19_numpy(10))
    print("task_19 loops:", task_19_loops(10))
    compare_system_solvers()
    compare_matrix_expr()


task_2: True
task_4:
 [[0 1 2 3]
 [0 0 0 0]
 [0 0 0 0]]
task_5:
 [[1 1 1 1 1]
 [0 0 0 0 0]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [1 1 1 1 1]]
task_8: (np.int32(3), np.int32(8))
task_9: [10  9  8  7  6  5  4  3  2  1  0]
task_12:
 [[0. 0. 0. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 0. 0. 0.]]
task_13:
 [[0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0]]
task_15:
 [[0 1 0 1 0]
 [0 1 0 1 0]
 [0 1 0 1 0]
 [0 1 0 1 0]
 [0 1 0 1 0]]
task_19 numpy: [0.083 0.167 0.25  0.333 0.417 0.5   0.583 0.667 0.75  0.833 0.917]
task_19 loops: [0.091 0.182 0.273 0.364 0.455 0.545 0.636 0.727 0.818 0.909]
Рішення (solve): [ 1.  2.  1. -1.]
inv@b збігається: True
Крамер збігається: True
Гаусс збігається : True
Час: inv=0.000000  solve=0.000000  cramer=0.000000  gauss=0.000100
allclose: True
Час: numpy=0.000000  loops=0.000000
Результат:
 [[ 138.   88.   68.]
 [-154. -100.  -78.]
 [ -14.  -16.  -12.]]
